# 4. Messages
- 모델과의 상호작용에서 가장 기본이 되는 단위입니다.
- 체인을 구성할 때는 `ChatPromptTemplate`를 사용합니다.

**메시지의 구성 요소**
| 요소 | 설명 | 예시 |
| :--- | :--- | :--- |
| **Role (역할)** | 메시지의 발신자 유형 | system, user, assistant, tool |
| **Content (내용)** | 실제 메시지 내용 | 텍스트, 이미지, 파일 등 |
| **Metadata** | 부가 정보 | 토큰 수, 응답 ID 등 |

## 메시지 유형

In [1]:
from langchain_core.messages import (
    SystemMessage,
    HumanMessage,
    AIMessage,
    ToolMessage,
)

### 1. SystemMessage
- 시스템 메시지는 모델의 행동 방식을 정의합니다. 대화의 맥락, 역할, 제약사항 등을 설정합니다.

In [2]:
from langchain_core.messages import SystemMessage

# 기본 시스템 메시지
system_msg = SystemMessage(content="당신은 친절한 한국어 AI 어시스턴트입니다.")

# 상세한 역할 정의
system_msg = SystemMessage(content=
                           """당신은 파이썬 프로그래밍 전문가입니다.
                           - 초보자도 이해할 수 있게 설명하세요
                           - 코드 예제를 포함하세요
                           - 한국어로 답변하세요""")

### 2. HumanMessage (사용자 메시지)
- 사용자의 입력을 나타냅니다. 텍스트뿐 아니라 이미지 등 멀티모달 콘텐츠를 포함할 수 있습니다.

In [3]:
from langchain_core.messages import HumanMessage

# 텍스트 메시지
human_msg = HumanMessage(content="파이썬 리스트 컴프리헨션을 설명해주세요.")

# 이름 포함 (선택)
human_msg = HumanMessage(
    content="안녕하세요!",
    name="user_123"
)

**멀티모달 입력**
- 이미지를 포함한 멀티모달 메시지를 구성할 수 있습니다. GPT-4o, Claude 3 등 비전 기능을 지원하는 모델에서 사용 가능합니다.

URL 기반 이미지 입력

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage

model = init_chat_model("gpt-4o-mini")

# 이미지 URL 포함
message = HumanMessage(
    content=[
        # 이미지 첨부 시 아래와 같은 형식으로 작성 필요
        {"type": "text", "text": "이 이미지에 무엇이 있나요?"},
        {"type": "image_url", 
         "image_url": {"url": "https://images.pexels.com/photos/45170/kittens-cat-cat-puppy-rush-45170.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500"}}
    ]
)

response = model.invoke([message])
print(response.content)

이미지에는 여러 마리의 아기 고양이들이 풀밭 위에 서 있는 모습이 있고, 다양한 색상의 털을 가지고 있습니다. 고양이들은 서로 가까이 모여 있으며, 귀여운 모습으로 주위를 살펴보고 있습니다.


**Base64 인코딩 이미지**
- 로컬 이미지를 Base64로 인코딩하여 전송할 수 있습니다.

In [14]:
import base64
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage

# 이미지 파일을 Base64로 인코딩
def encode_image(image_path: str) -> str:
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

image_data = encode_image("E:\Portfolio\Study\hello-langchain\KakaoTalk_20251208_143807859.jpg")

model = init_chat_model("gpt-4o-mini")

message = HumanMessage(
    content=[
        {"type": "text", "text": "이 이미지에 무엇이 있나요?"},
        {"type": "image_url",
         "image_url":
         {"url": f"data:image/png;base64,{image_data}"}
         }
    ]
)

response = model.invoke([message])
print(response.content)

이 이미지는 일본의 전통적인 도리이 기둥 구조물로 보입니다. 도리이는 일반적으로 신사의 입구를 나타내며, 주로 붉은색으로 칠해져 있습니다. 배경에는 산과 물이 보이며, 전반적으로 평화로운 풍경을 이루고 있습니다.


**여러 이미지 동시 처리**

In [ ]:
message = HumanMessage(
    content=[
        {"type": "text", "text": "두 이미지를 비교해주세요."},
        {
            "type": "image_url",
            "image_url": {"url": "https://example.com/image1.jpg"}
        },
        {
            "type": "image_url",
            "image_url": {"url": "https://example.com/image2.jpg"}
        }
    ]
)

**이미지 상세도 설정**
상세도 | 특징 |	토큰 사용량
| :--- | :--- | :--- |
low | 빠른 처리, 저해상도 |	낮음
high | 고해상도 분석 | 높음
auto | 자동 선택 (기본값) | 상황별

In [16]:
message = HumanMessage(
    content=[
        {"type": "text", "text": "이 문서를 읽어주세요."},
        {
            "type": "image_url",
            "image_url": {
                "url": "https://example.com/document.png",
                "detail": "high"  # low, high, auto
            }
        },
    ]
)


### 3. AIMessage
- 모델의 응답을 나타냅니다. 모델 호출 결과로 반환되며, 다양한 메타데이터를 포함합니다.

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import AIMessage

model = init_chat_model("gpt-4o-mini")

# 모델 호출 - AIMessage 반환
response = model.invoke([HumanMessage(content="안녕하세요!")])

print(type(response))
print(response.content)

<class 'langchain_core.messages.ai.AIMessage'>
안녕하세요! 어떻게 도와드릴까요?


In [ ]:
response = model.invoke([HumanMessage(content="안녕!")])

# AIMessage 속성
print(f"내용: {response.content}")
print(f"토큰 사용량: {response.usage_metadata}")
print(f"응답 메타데이터: {response.response_metadata}")

내용: 안녕하세요! 어떻게 도와드릴까요?
토큰 사용량: {'input_tokens': 10, 'output_tokens': 10, 'total_tokens': 20, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
응답 메타데이터: {'token_usage': {'completion_tokens': 10, 'prompt_tokens': 10, 'total_tokens': 20, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-D0KRU7OuDJO9pWedehWBcoJ8xw8pj', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}


**Tool Calls (도구 호출)**
- AIMessage는 도구 호출 정보를 포함할 수 있습니다.

In [19]:
if response.tool_calls:
    for tool_call in response.tool_calls:
        print(f"도구 이름: {tool_call['name']}")
        print(f"도구 인자: {tool_call['args']}")

### 4. ToolMessage (도구 메시지)
- 도구 실행 결과를 모델에 전달할 때 사용합니다.

In [20]:
from langchain_core.messages import ToolMessage

tool_msg = ToolMessage(
    content="서울의 현재 온도는 25도입니다.",
    tool_call_id = "call_abc123",
    name="get_weather"
)

### 기타
**메시지 리스트로 대화 구성**
- 모델에 메시지 리스트를 전달하여 대화 컨텍스트를 구성합니다.

In [21]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

model = init_chat_model("gpt-4o-mini")

# 대화 히스토리 구성
messages = [
    SystemMessage(content="당신은 요리 전문가입니다."),
    HumanMessage(content="김치찌개 만드는 법을 알려주세요."),
    AIMessage(content="김치찌개는 다음과 같이 만듭니다..."),
    HumanMessage(content="고기 없이 만들 수 있나요?")
]

# 컨텍스트를 유지한 응답
response = model.invoke(messages)
print(response.content)

네, 고기 없이도 맛있는 김치찌개를 만들 수 있습니다! 아래는 고기 없는 김치찌개 레시피입니다.

### 재료:
- 잘 익은 김치 2컵 (국물이 있는 김치가 좋습니다)
- 물 3컵
- 두부 1/2모 (먹기 좋은 크기로 자른 것)
- 양파 1개 (슬라이스)
- 대파 1대 (어슷 썬 것)
- 마늘 2~3쪽 (다진 것)
- 고춧가루 1~2큰술 (취향에 따라 조절)
- 간장 1~2큰술 (맛 조절)
- 소금 및 후추 (간을 맞추기 위해)
- 원하는 부재료 (애호박, 시금치 등) - 선택사항

### 조리 방법:
1. **Preparation**: 냄비에 잘 익은 김치를 넣고 중불에서 약간 볶아줍니다. 이 과정에서 김치의 풍미가 더욱 살아납니다.

2. **Add Water**: 김치가 어느 정도 볶아지면 물을 부어줍니다. 그리고 끓이기 시작합니다.

3. **Add Vegetables**: 물이 끓으면 양파, 대파, 다진 마늘을 넣어줍니다. 선택한 부재료가 있다면 이 시점에 함께 넣어줍니다.

4. **Seasoning**: 고춧가루와 간장을 넣고, 한 번 더 끓입니다. 간이 부족하면 소금과 후추로 추가 간을 맞춥니다.

5. **Tofu**: 마지막으로 두부를 넣고 5~10분 정도 더 끓입니다. 두부가 잘 익으면 완성입니다.

6. **Serve**: 뜨거운 상태에서 밥과 함께 서빙합니다.

맛있게 드세요! 고기 없이도 풍부한 맛을 내는 김치찌개가 됩니다.


**딕셔너리 형식**
- 메시지 객체 대신 딕셔너리 형식도 사용할 수 있습니다.

방식 | 장점 | 사용 시점
| :--- | :--- | :--- |
Message 객체 | 타입 안전성, IDE 자동완성 | 프로덕션 코드
딕셔너리 | 간결함, JSON 호환 | 빠른 프로토타이핑

In [22]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o-mini")

# 딕셔너리 형식의 메시지
messages = [
    {"role": "system", "content": "당신은 유용한 AI 어시스턴트입니다."},
    {"role": "user", "content": "파이썬이란?"},
    {"role": "assistant", "content": "파이썬은 프로그래밍 언어입니다."},
    {"role": "user", "content": "주요 특징은?"},
]

response = model.invoke(messages)
print(response.content)

파이썬의 주요 특징은 다음과 같습니다:

1. **간결하고 읽기 쉬운 문법**: 파이썬은 코드의 가독성이 높아 배우기 쉽고, 유지보수가 용이합니다.

2. **인터프리터 언어**: 코드가 실행되기 전에 컴파일할 필요 없이 바로 실행할 수 있어, 디버깅이 용이합니다.

3. **다양한 라이브러리와 프레임워크**: 파이썬은 수많은 라이브러리와 프레임워크를 제공하여 데이터 분석, 웹 개발, 머신러닝 등 다양한 분야에서 활용됩니다.

4. **객체 지향 및 함수형 프로그래밍 지원**: 파이썬은 객체 지향 프로그래밍(OOP)과 함수형 프로그래밍(FP) 패러다임을 모두 지원합니다.

5. **플랫폼 독립성**: 다양한 운영 체제에서 실행할 수 있어, 코드의 이식성이 높습니다.

6. **활발한 커뮤니티와 생태계**: 파이썬은 큰 사용자 커뮤니티가 있어, 자료와 도움을 쉽게 찾을 수 있습니다.

7. **다양한 용도**: 웹 개발, 데이터 분석, 인공지능, 머신러닝, 자동화 스크립트 작성 등 다양한 분야에서 사용됩니다.

이러한 특징 덕분에 파이썬은 초보자부터 전문가까지 널리 사용되는 언어가 되었습니다.


### 요약

메시지 유형	| 역할 | 주요 용도
| :--- | :--- | :--- |
SystemMessage | system | 모델 행동 지시, 역할 정의
HumanMessage | user | 사용자 입력, 질문, 멀티모달 데이터
AIMessage | assistant | 모델 응답, 도구 호출
ToolMessage | tool | 도구 실행 결과 전달

### 메시지 사용 팁

**기본 사용**

- 시스템 메시지는 대화 시작 시 한 번 설정

- 대화가 길어지면 오래된 메시지를 요약하거나 삭제

- 빠른 테스트는 딕셔너리, 프로덕션은 메시지 객체 사용

**멀티모달**

- 이미지 분석이 필요하면 GPT-4o, Claude 3 등 비전 지원 모델 사용

- 로컬 이미지는 Base64 인코딩 후 `data:image/...;base64,` 형식으로 전송

- 문서 분석 시 `detail: "high"` 옵션 사용